In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import math
import ROOT
import numpy as np, pandas as pd
import yaml
import base64
import ctypes
import logging

import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file

from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
from RootInteractive.Tools.aliTreePlayer import *

output_notebook()

# Display skimmed AO2D tracks

In [ ]:
inputFile="/lustre/alice/users/miranov/NOTESData/alice-tpc-notes/JIRA/ATO-592/tpcqcskimmingTracks.root"
#inputFile="~/Software/TPCQCVis/data/examples/SkimmedTracks/tpcqcskimmingTracks.root"

In [ ]:
f=ROOT.TFile.Open(inputFile)
tracks = f.Get("tracks;2")  
tracks.SetAlias("weight1","((triggerMask&0x1)>0)*max(weight,0.01)")
tracks.SetAlias("rMin","trackPar.mX")
tracks.SetAlias("qPt","trackPar.mP[4]")
tracks.SetAlias("pzPt","trackPar.mP[3]")
tracks.SetAlias("itsOn","itsClusterMap>0")
tracks.SetAlias("tofOn","abs(tofChi2)<100")
#tracks.SetAlias("side","pzPt>0")

#tracks.Show(0)

In [ ]:
tracks.Show(0)

In [ ]:
df=tree2Panda(tracks, [".*"], "", exclude=["trackPar"])
df["side"]=df.eval("pzPt>0") ##  (A/C/CrossAC/CrossCA side)
df["isPrim5"]=df.eval("abs(dcaXY)<5 &  abs(dcaZ)<5 & rMin<5")

df.head()

In [ ]:
widgetParams=[['range', ['pt']],
              ['range', ['qPt']],
              ['range', ['phi']],
              ['range', ['pzPt']],
              #
              ['range', ['dcaXY']],
              ['range', ['dcaZ']],
              ['range', ['rMin']],
              ['range', ['tpcNClsFindable']],
              #
              ['multiSelect',["side"]],
              ['multiSelect',["isPrim5"]],
              ['multiSelect',["itsOn"]],
              ['multiSelect',["tofOn"]],
              ['multiSelect',["hasCollision"]],
             ]
widgetLayoutDesc=[[0,1,2,3],[4,5,6,7],[8,9,10,11,12], {'sizing_mode': 'scale_width'}]   
#widgetLayoutDesc=[[0,1,2,3],[4,5,6], {'sizing_mode': 'scale_width'}]  
#widgetLayoutDesc=[[0],[1], {'sizing_mode': 'scale_width'}] 

In [ ]:
aliasArray = [    {
        "name": "logPtWeight",
        "variables": ["bin_count"],
        "func": "return Math.log(bin_count)",
        "context": "ptWeightedHist"
    }
]


In [ ]:
output_file("investigating_skimmed_tracks.html")
histoArray = [{"name": "ptHist", "variables": ["pt"], "nbins":200, "range": [0, 20]},
              {"name": "ptWeightedHist", "variables": ["pt"], "nbins":200, "range": [0, 20], "weights":"weight"},
              {"name": "weightHist", "variables": ["weight"], "nbins":200, "range": [0, 20]},
              {"name": "ptWeightHist", "variables": ["pt","weight"], "nbins":[200,200], "range": [[0, 20],[0,20]]},
              {"name": "phiWeightedHist", "variables": ["phi"], "nbins":180, "range": [0, 6.28], "weights":"weight"},
              {"name": "phiHist", "variables": ["phi"], "nbins":180, "range": [0, 6.28]},
              {"name": "phiHistA", "variables": ["phi"], "nbins":90},
              {"name": "ptVsPhi", "variables": ["pt","phi"], "nbins":[20,18], "range": [[0,20],[0, 6.28]]}]

figureArray = [[['pt'],['ptHist']],
               [['pt'],['ptWeightedHist']],
               [['bin_center'],['logPtWeight'],{"source":"ptWeightedHist"}],
               [['pt'],['ptVsPhi']],
               [['phi'],['phiWeightedHist','phiHist']],
               [['phi'],['phiHistA']],
              ]

layout = [[0,2],[3,5]]
tooltips = []

  
xxx = bokehDrawSA.fromArray(df, "(triggerMask%2)==1&pt<20&abs(qPt)<20", figureArray, widgetParams, layout=layout, tooltips=tooltips, widgetLayout=widgetLayoutDesc,
                            histogramArray=histoArray,aliasArray=aliasArray)

In [ ]:
c1=ROOT.TCanvas("c","c")
tracks.SetAlias("itsOn","(flags&0x1)>0")
tracks.Draw("tpcNClsFindable","")
c1.Draw()

In [ ]:
tracks.GetLeaf("tpcNClsFindable").Print()
tracks.GetLeaf("tpcNClsFindableMinusFound").Print()

In [ ]:
c1=ROOT.TCanvas("c","c")
tracks.SetAlias("itsOn","(flags&0x1)>0")
tracks.Draw("dcaXY:phi","pzPt<0&&phi<1&&abs(dcaXY)<10&&pt>2","prof")
c1.Draw()